In [ ]:
from IPython.display import display, HTML

%run -i /notebooks/helper/aws.py
%run -i /notebooks/helper/aws_functions.py
aws_ctx = await aws_connect(
    profiles=[
        "sandbox",
    ],
    regions=[
        "us-east-1",
    ],
    svc_include=[
        "ssm",
    ],
)


def get_all_env_parameters(ssm_client, env_id):
    pager = ssm_client.get_paginator("get_parameters_by_path")
    params_response = pager.paginate(
        Path=f"/DGC/{env_id}", Recursive=True
    ).build_full_result()
    env_parameters = params_response.get("Parameters")
    decrypted_env_params = {
        param.get("Name"): ssm_client.get_parameter(
            Name=param.get("Name"), WithDecryption=True
        )
        .get("Parameter", {})
        .get("Value")
        for param in env_parameters
    }
    return decrypted_env_params

In [ ]:
env_params = get_all_env_parameters(aws_ctx.sandbox.us_east_1.ssm, "1b3f969e-ed7c-5240-8a89-6ed164defe8e")

In [ ]:
html_src = ""
for name, value in env_params.items():
    html_src += f"<h4>{name}</h4><div>{value}</div>"
display(HTML(html_src))